In [1]:
from pathlib import Path
import sys
import os
import json
from tqdm import tqdm
from transformers import AutoTokenizer
import importlib
import difflib
from collections import defaultdict

import context
os.chdir(context.proj_dir)

import cont_gen
import cont_gen.data_process.ood.build_src_tgt
importlib.reload(cont_gen.data_process.ood.build_src_tgt)
from cont_gen.data_process.ood.build_src_tgt import process, SFT_Builder, SFT_Builder_YesNo, SFT_Builder_YesNo_Natural
from cont_gen.utils import load_jsonl, save_jsonl

## Overview

For each tokenizer and prompt method, build source and target data and save to `data/ood_split/{split_name}/{prompt_name}/{tokenizer short name}`